# Importing Libraries and Packages

In [1]:
# Install required packages for Albert model
!pip install -q sentencepiece
!pip install -q transformers
!pip install -q tokenizers
!pip install -qU hazm
!pip install -qU clean-text[gpl]


#!pip install git+https://github.com/LIAAD/yake

!pip install rank_bm25

!mkdir resources
!wget -q "https://github.com/sobhe/hazm/releases/download/v0.5/resources-0.5.zip" -P resources
!unzip -qq resources/resources-0.5.zip -d resources


!rm -rf /content/4ccae468eb73bf6c4f4de3075ddb5336
!rm -rf /content/preproc
!rm preprocessing.py utils.py
!mkdir -p /content/preproc
!git clone https://gist.github.com/4ccae468eb73bf6c4f4de3075ddb5336.git /content/preproc/
!mv /content/preproc/* /content/
!rm -rf /content/preproc




     |████████████████████████████████| 1.2MB 5.4MB/s 
     |████████████████████████████████| 1.8MB 5.9MB/s 
     |████████████████████████████████| 3.2MB 46.0MB/s 
     |████████████████████████████████| 890kB 48.8MB/s 
     |████████████████████████████████| 317kB 5.6MB/s 
     |████████████████████████████████| 235kB 7.1MB/s 
     |████████████████████████████████| 1.4MB 8.7MB/s 
     |████████████████████████████████| 133kB 4.1MB/s 
     |████████████████████████████████| 71kB 8.2MB/s 
     |████████████████████████████████| 245kB 27.7MB/s 
rm: cannot remove 'preprocessing.py': No such file or directory
rm: cannot remove 'utils.py': No such file or directory
Cloning into '/content/preproc'...
remote: Enumerating objects: 7, done.
remote: Total 7 (delta 0), reused 0 (delta 0), pack-reused 7
Unpacking objects: 100% (7/7), done.


In [2]:
import numpy as np 
import pandas as pd
import re
from tqdm import tqdm 
import os

from hazm import *
import pickle
import requests
from termcolor import colored
from preprocessing import cleaning
from rank_bm25 import BM25Okapi
import time


import plotly.express as px
import plotly.graph_objects as go
from itertools import chain
# for the models
import tensorflow as tf
import matplotlib.pyplot as plt
from transformers import BertConfig, BertTokenizer
Base_BERT_Path = 'HooshvareLab/bert-fa-base-uncased'

# Loading the dataset


In [5]:
from google.colab import drive
drive.mount('/content/drive')

data_address = '/content/drive/MyDrive/COVID-PSS.xls'
keys_address = '/content/drive/MyDrive/keywords_final_distilled_NE (1).pickle'
cleaned_titles_address = '/content/drive/MyDrive/title_cleaned_without_corona_2.pkl'


df = pd.read_csv(data_address)
list_t = pd.read_pickle(cleaned_titles_address)

keywords = pd.read_pickle(keys_address)
keywords = [v for k,v in keywords.items()]



assert len(keywords) == len(df)
df['keywords'] = keywords
df.drop(columns=['img', 'link'], inplace=True)

Mounted at /content/drive


# Preparing Inputs for BM25

In [6]:
bert_tokenizer = BertTokenizer.from_pretrained(Base_BERT_Path)

normalizer = Normalizer()

# joining title to body to show the whole doc to BM25
corpora = []
for i in range(len(df)):
    corpora.append(' '.join([df.loc[i, 'title'], df.loc[i, 'body']]))


# changed word_tokenize to bert tokenizer....................
tokenized_corpus = [bert_tokenizer.tokenize(normalizer.normalize(each)) for each in corpora]

bm25 = BM25Okapi(tokenized_corpus)

# Ranking results

In [7]:
questions = pd.read_pickle('/content/drive/MyDrive/CoPer paper-Models/Sample Queries/Titles_with_Corona.pkl')

In [8]:
print(questions[:1][0])

آیا زنان باردار بیشتر در معرض خطر ابتلا به بیماری کروناهستند 


In [9]:
def rank(df, questions, top_x_percent=None):
    """Gets the df and questions and returns a percentage of the dataset indices it finds relevant
    if top_x_percent == None it'll return all the relevents with score>0
    """

    first_ranking = []
    for question in questions:
        print(question)
        query = bert_tokenizer.tokenize(question)
        doc_scores = bm25.get_scores(query)

        if top_x_percent !=None:
            pivot = 1 * int((top_x_percent * len(doc_scores)) / 100)
            least_score = sorted(doc_scores)[-1 * pivot]

            print(f'The top {top_x_percent} percent are records with the scores more than', colored(np.round(least_score, 2), 'blue'))
        else:
            least_score = 0

        bm_selected = []
        for i in range(len(df)):
            if doc_scores[i] > least_score : 
                bm_selected.append(i)

        print(len(bm_selected), 'selected.')

        first_ranking.append({'question':question,
                              'index':bm_selected})
    return first_ranking

In [10]:
first_ranking = rank(df, questions, top_x_percent=99.5)

آیا زنان باردار بیشتر در معرض خطر ابتلا به بیماری کروناهستند 
The top 99.5 percent are records with the scores more than 7.58
3517 selected.
من باردار هستم چگونه می‌توانم از خودم در مقابل بیماری کرونامحافظت کنم
The top 99.5 percent are records with the scores more than 7.28
3517 selected.
آیا زنان باردار باید برای کروناآزمایش شوند
The top 99.5 percent are records with the scores more than 0.0
3516 selected.
آیا کرونامی تواند از مادر باردار به نوزاد متولد نشده جنین یا نوزاد تازه متولد شده منتقل شود
The top 99.5 percent are records with the scores more than 9.17
3517 selected.
در دوران بارداری حین زایمان چه مراقبت‌هایی باید انجام شود
The top 99.5 percent are records with the scores more than 3.22
3517 selected.
آیا زنان باردار مبتلا به کرونایا مشکوک به این بیماری نیاز به سزارین دارند
The top 99.5 percent are records with the scores more than 11.07
3517 selected.
آیا مادر مبتلا به کرونامی تواند به نوزاد خود شیر بدهد
The top 99.5 percent are records with the scores more than 7.63
3517 sele

# Normal BM25

In [11]:
def get_bm25_results(df, questions, top_n=50):
    """Gets the df and questions and returns a percentage of the dataset indices it finds relevant"""

    bm_results = []
    for question in questions:
        print(question)
        query = bert_tokenizer.tokenize(question)
        doc_scores = bm25.get_scores(query)

        indices = bm25.get_top_n(query, df.index, n=top_n)
        

        bm_results.append({'question':question,
                            'index':indices})
    return bm_results

In [12]:
bm_results = get_bm25_results(df, questions, top_n=50)

آیا زنان باردار بیشتر در معرض خطر ابتلا به بیماری کروناهستند 
من باردار هستم چگونه می‌توانم از خودم در مقابل بیماری کرونامحافظت کنم
آیا زنان باردار باید برای کروناآزمایش شوند
آیا کرونامی تواند از مادر باردار به نوزاد متولد نشده جنین یا نوزاد تازه متولد شده منتقل شود
در دوران بارداری حین زایمان چه مراقبت‌هایی باید انجام شود
آیا زنان باردار مبتلا به کرونایا مشکوک به این بیماری نیاز به سزارین دارند
آیا مادر مبتلا به کرونامی تواند به نوزاد خود شیر بدهد
آیا در صورت مبتلا شدن به کووید۱۹ می‌توانم کودک خود را در آغوش بگیرم
من به کرونامبتلا هستم حال عمومی مساعدی ندارم که بتوانم به کودک خود مستقیما شیر بدهم چه باید بکنم
آیا خشک­ کن­های دست در نابودی ویروس جدید کرونا موثر هستند
آیا گندزداییضدعفونی با لامپ ماورای بنفش ویروس جدید کرونا را از بین می­برد
آیا سنجش حرارت بدن اندازه گیری تب در شنسایی افراد مبتلا به عفونت ویروس جدید کرونا موثر است
آیا اسپری کردن الکل یا محلول‌های حاوی کلر روی تمام بدن می­تواند ویروس جدید کرونا را نابود کند
آیا واکسنهای ذات­الریه در برابر ویروس جدید کرونا موثر هستند
آیا ش

# Saving the Results

In [13]:
with open('/content/drive/MyDrive/CoPer paper-Models/Results/BM25.pkl', 'wb') as f:
    pickle.dump(bm_results, f)

with open('/content/drive/MyDrive/CoPer paper-Models/Results/BM25_PrimaryRanking.pkl', 'wb') as f:
    pickle.dump(first_ranking, f)

